In [ ]:
import os
import pandas as pd

# Folder containing all your CSV files
base_folder_2020 = "/Users/parthpatel/Mtech/DS_Prac_Project/Soil_data/2020"
base_folder_2018 = "/Users/parthpatel/Mtech/DS_Prac_Project/Soil_data/2018"

def process_csv(csv_path):
    print(f"\n➡ Processing: {csv_path}")

    df = pd.read_csv(csv_path)

    # Standardize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # If all columns exist -> skip file
    if all(col in df.columns for col in ["year", "month", "day"]):
        print("✔ Already has Year/Month/Day → Skipping")
        return

    # Detect date column
    date_cols = [c for c in df.columns if "date" in c]
    if not date_cols:
        print("❌ No date column found → skipping")
        return

    date_col = date_cols[0]
    print(f"   Using date column: {date_col}")

    # Parse date safely
    df["date_parsed"] = pd.to_datetime(
        df[date_col],
        errors="coerce",
        infer_datetime_format=True,
        dayfirst=True
    )

    # Create new columns
    df["Year"] = df["date_parsed"].dt.year
    df["Month"] = df["date_parsed"].dt.strftime("%b")  # Jan, Feb, Mar
    df["Day"] = df["date_parsed"].dt.day

    # Drop helper column
    df.drop(columns=["date_parsed"], inplace=True)

    # Save back to SAME file
    df.to_csv(csv_path, index=False)
    print("✔ Updated Year / Month / Day added.")

def scan_and_process_all(folder):
    print("\n🔍 Scanning for CSV files...\n")
    
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root, file)
                process_csv(csv_path)

    print("\n🎉 ALL FILES PROCESSED SUCCESSFULLY!")

# Run the scanner
scan_and_process_all(base_folder_2018)
scan_and_process_all(base_folder_2020)


In [11]:
# Combined merge + inspect + produce final merged_2018_final.csv in one cell (no intermediate per-year CSV)
import re
import pandas as pd
from pathlib import Path
root = Path('/Users/parthpatel/Mtech/DS_Prac_Project')
out2018_final = root / 'merged_2018_final.csv'
out2020_final = root / 'merged_2020_final.csv'
out_combined = root / 'merged_final.csv'

def load_folder_frames(folder):
    folder = Path(folder)
    frames = []
    for f in sorted(folder.glob('*.csv')):
        try:
            df = pd.read_csv(f, encoding='utf-8', on_bad_lines='skip')
        except Exception:
            df = pd.read_csv(f, encoding='latin1', on_bad_lines='skip')
        df['source_file'] = f.name
        s = re.sub(r'^sm[_\-]*','', f.name, flags=re.I)
        s = re.sub(r'[_\-]*\d{4}\.csv$','', s, flags=re.I)
        state_from_file = s.replace('_',' ').replace('.csv','').strip()
        state_col = None
        for cand in ['State Name','State','state','state_name','StateName']:
            if cand in df.columns:
                state_col = cand
                break
        if state_col:
            df['State'] = df[state_col]
        else:
            df['State'] = None
        # fill missing State with inferred name
        df['State'] = df['State'].fillna(state_from_file)
        # drop rows that are entirely empty
        df = df.dropna(how='all')
        frames.append(df)
    return frames

# Collect frames for 2018 and combine, then directly create merged_2018_final.csv
print('Loading 2018 CSVs and building final merged_2018_final.csv...')
frames2018 = load_folder_frames(root / '2018')
if frames2018:
    df2018 = pd.concat(frames2018, ignore_index=True)
    # choose canonical state column
    if 'State Name' in df2018.columns:
        state_col = 'State Name'
    elif 'State' in df2018.columns:
        state_col = 'State'
    else:
        state_col = 'State'  # we ensured df['State'] exists from filename fallback
    # drop rows missing state in the canonical column
    df2018_cleaned = df2018.dropna(subset=[state_col])
    # remove helper columns and write final file
    columns_to_drop = [c for c in ['source_file'] if c in df2018_cleaned.columns]
    df2018_final = df2018_cleaned.drop(columns=columns_to_drop)
    df2018_final.to_csv(out2018_final, index=False)
    print('Wrote', out2018_final, 'shape', df2018_final.shape)
else:
    print('No 2018 CSVs found in', root / '2018')

# Optionally process 2020 similarly and write combined merged_final.csv
print('Loading 2020 CSVs...')
frames2020 = load_folder_frames(root / '2020')
if frames2020:
    df2020 = pd.concat(frames2020, ignore_index=True)
    # ensure state column exists and drop rows missing it
    if 'State Name' in df2020.columns:
        state_col_2020 = 'State Name'
    elif 'State' in df2020.columns:
        state_col_2020 = 'State'
    else:
        state_col_2020 = 'State'
    df2020_cleaned = df2020.dropna(subset=[state_col_2020])
    columns_to_drop_2020 = [c for c in ['source_file'] if c in df2020_cleaned.columns]
    df2020_final = df2020_cleaned.drop(columns=columns_to_drop_2020)
    df2020_final.to_csv(out2020_final, index=False)
    print('Wrote', out2020_final, 'shape', df2020_final.shape)
else:
    print('No 2020 CSVs found in', root / '2020')

# Create combined merged_final.csv from the two final dataframes if both exist
combined_frames = []
if (root / out2018_final.name).exists():
    combined_frames.append(pd.read_csv(root / out2018_final.name))
if (root / out2020_final.name).exists():
    combined_frames.append(pd.read_csv(root / out2020_final.name))
if combined_frames:
    merged_all = pd.concat(combined_frames, ignore_index=True)
    merged_all.to_csv(out_combined, index=False)
    print('Wrote combined merged_final.csv shape', merged_all.shape)
else:
    print('No final per-year files found to combine into merged_final.csv')


Loading 2018 CSVs and building final merged_2018_final.csv...
Wrote /Users/parthpatel/Mtech/DS_Prac_Project/merged_2018_final.csv shape (265788, 11)
Loading 2020 CSVs...
Wrote /Users/parthpatel/Mtech/DS_Prac_Project/merged_2020_final.csv shape (255626, 11)


/var/folders/6_/strprdxj5c17tqpj_l4g1m840000gn/T/ipykernel_69491/2343962776.py:85: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_frames.append(pd.read_csv(root / out2020_final.name))


Wrote combined merged_final.csv shape (521414, 11)


In [19]:
#get Unique  names from merger_final col State
#Load merged_final.csv

df = pd.read_csv('/Users/parthpatel/Mtech/DS_Prac_Project/merged_final.csv')
unique_states = df['state_name'].unique()
print(len(unique_states))
print(unique_states)


38
['ASSAM' 'BIHAR' 'CHANDIGARH' nan 'CHHATTISGARH' 'DADRA AND NAGAR HAV'
 'DAMAN & DIU' 'DELHI' 'GOA' 'GUJARAT' 'HARYANA' 'HIMACHAL PRADESH'
 'JAMMU & KASHMIR' 'JHARKHAND' 'KARNATAKA' 'KERALA' 'LAKSHDWEEP'
 'MADHYA PRADESH' 'MAHARASHTRA' 'MANIPUR' 'MEGHALAYA' 'MIZORAM' 'NAGALAND'
 'ODISHA' 'PONDICHERRY' 'PUNJAB' 'RAJASTHAN' 'SIKKIM' 'TAMILNADU'
 'TELANGANA' 'TRIPURA' 'UTTARAKHAND' 'UTTAR PRADESH' 'WEST BENGAL'
 'ANDAMAN & NICOBAR' 'ANDHRA PRADESH' 'ARUNACHAL PRADESH' 'Ladakh']


/var/folders/6_/strprdxj5c17tqpj_l4g1m840000gn/T/ipykernel_69491/2915988445.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/parthpatel/Mtech/DS_Prac_Project/merged_final.csv')


In [20]:
state_season_map = {

    # ====== SOUTH INDIA ======
    "ANDHRA PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TELANGANA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TAMILNADU": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },
    "KARNATAKA": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },
    "KERALA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "LAKSHDWEEP": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "PONDICHERRY": {
        "Winter": ["JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV", "DEC"]
    },

    # ====== NORTH INDIA ======
    "DELHI": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "PUNJAB": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "HARYANA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "HIMACHAL PRADESH": {
        "Winter": ["NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT"]
    },
    "UTTARAKHAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "UTTAR PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "JAMMU & KASHMIR": {
        "Winter": ["NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT"]
    },
    "Ladakh": {
        "Winter": ["OCT", "NOV", "DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG"],
        "Post-monsoon": ["SEP"]
    },
    "CHANDIGARH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== EAST INDIA ======
    "WEST BENGAL": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "BIHAR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "JHARKHAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "ODISHA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== WEST INDIA ======
    "GUJARAT": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "RAJASTHAN": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "MAHARASHTRA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "GOA": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "DADRA AND NAGAR HAV": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "DAMAN & DIU": {
        "Winter": ["DEC", "JAN"],
        "Summer": ["FEB", "MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== CENTRAL INDIA ======
    "MADHYA PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "CHHATTISGARH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR", "MAY"],
        "Monsoon": ["JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },

    # ====== NORTHEAST INDIA ======
    "ASSAM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "ARUNACHAL PRADESH": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "MANIPUR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "MEGHALAYA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "MIZORAM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "NAGALAND": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },
    "SIKKIM": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP"],
        "Post-monsoon": ["OCT", "NOV"]
    },
    "TRIPURA": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },

    # ====== ISLANDS ======
    "ANDAMAN & NICOBAR": {
        "Winter": ["DEC", "JAN", "FEB"],
        "Summer": ["MAR", "APR"],
        "Monsoon": ["MAY", "JUN", "JUL", "AUG", "SEP", "OCT"],
        "Post-monsoon": ["NOV"]
    },

    None: {
        "Winter": [],
        "Summer": [],
        "Monsoon": [],
        "Post-monsoon": []
    }
}

In [21]:
df["Month"] = df["Month"].str.upper().str.strip()

In [22]:
# ---------------------
# Add Season Column
# ---------------------

def get_season(state, month):
    """
    Returns the season (Winter, Summer, Monsoon, Post-monsoon) 
    based on state and month abbreviation.
    """
    if pd.isna(state) or pd.isna(month):
        return "Unknown"

    state = state.upper().strip()
    month = month.upper().strip()

    if state not in state_season_map:
        return "Unknown"

    season_info = state_season_map[state]

    for season_name, month_list in season_info.items():
        if month in month_list:
            return season_name

    return "Unknown"

# Apply mapping
df["Season"] = df.apply(lambda row: get_season(row["state_name"], row["Month"]), axis=1)

print(df[["state_name", "Month", "Season"]].head())

  state_name Month   Season
0      ASSAM   JUN  Monsoon
1      ASSAM   JUN  Monsoon
2      ASSAM   JUN  Monsoon
3      ASSAM   JUN  Monsoon
4      ASSAM   JUN  Monsoon
